# Asymptotic Behaviour
---

Description:

    This notebook performs the asymptotic analysis of the algorthm:
    
    1) as the number of particles increase
    2) as the number of nearest neighbors increase
    
---

### Import python libraries and set up the project directory

In [ ]:
import json
import os, sys
from collections import defaultdict

import numpy as np
from matplotlib import pyplot as plt

# Enable LaTex in plotting.
plt.rcParams["text.usetex"] = True

PROJECT_DIR = os.path.abspath('..')

if PROJECT_DIR not in sys.path:
    sys.path.append(PROJECT_DIR)

### Import the custom PSO code

In [ ]:
from star_pso.population.swarm import Swarm
from star_pso.population.particle import Particle
from star_pso.engines.standard_pso import StandardPSO
from star_pso.utils.auxiliary import cost_function

### Load all the Benchmark problems

In [ ]:
# Dimensions: 1D
from star_pso.benchmarks.equal_maxima import EqualMaxima

# Dimensions: 2D
from star_pso.benchmarks.shubert import Shubert

In [ ]:
# Select one test problem.
benchmark = Shubert(n_dim=2)

# Make sure we don't run accidentally the wrong experiment.
# Options are: '01', '02'
EXPERIMENT_No = "01"

### Define the cost function

Note: All the optimization problems are solved as maximization.

In [ ]:
@cost_function(minimize=False)
def func_benchmark(x_array: np.ndarray, **kwargs) -> float:
    # Return the solution.
    return benchmark.func(x_array).item()
# _end_def_

### Setup the experiment 01

In [ ]:
if EXPERIMENT_No == "01":
    # List all the possible swarm sizes.
    swarm_size = [100, 200, 300, 400, 500, 600]
    
    # Define the number of repetitions.
    N_REPEATS = 50
    
    # Maximum function evaluations.
    MAX_F_EVALS = 200_000
    
    # Stores the statistics.
    stats_per_run = defaultdict(list)
    
    # Set up the main loop.
    for n_size in swarm_size:
    
        print(f"Running simulation with swarm size={n_size} ... \n", flush=True)
    
        for k in range(N_REPEATS):
    
            print(f"Repetition {k} in process ... ", flush=True)
    
            # Get an initial (new) sample.
            x_sample = benchmark.sample_random_positions(n_pos=n_size)
    
            # Initial swarm population.
            swarm_t0 = Swarm([Particle(x) for x in x_sample])
        
            # Create the StandardPSO object that will carry on the optimization.
            test_PSO = StandardPSO(initial_swarm = swarm_t0, obj_func = func_benchmark,
                                   x_min = benchmark.x_min, x_max = benchmark.x_max)
            # Run the experiment.
            test_PSO.run(max_it = 500, f_max_eval = MAX_F_EVALS, verbose = False,
                         options = {"w0": 0.70, "c1": 1.50, "c2": 1.50, "mode": "multimodal"})
    
            # Obtain the number of the global optima that were found for epsilon = 1.0e-4.
            gopt_found, gopt_total = benchmark.search_for_optima(test_PSO.swarm.best_n(n_size))
    
            # Update the dictionary.
            stats_per_run[n_size].append(gopt_found)
            
            # Leave a blank line between repetitions.
            print(" ")
        # _end_for_
    # _end_for_
    
    # Save the results.
    with open(f"E01_result_{benchmark.name}.json", 'w') as fp:
        json.dump(stats_per_run, fp)
    
    # Display final message.
    print(f"Experiment FINISHED.")

    # Get the data (counts).
    labels, data = stats_per_run.keys(), stats_per_run.values()

    # Prepare the PLOT.
    plt.boxplot(data)
    plt.xticks(range(1, len(labels) + 1), labels)
    
    plt.title(f"{benchmark.name}: No. Global optima = {gopt_total}")
    plt.xlabel("Swarm size")
    plt.ylabel("Global optima found")
    plt.grid()
    
    # Save to file.
    plt.savefig(f"E01_{benchmark.name}.pdf", format="pdf", dpi=300)
# _end_if_

### Setup Experiment 02

In [ ]:
if EXPERIMENT_No == "02":
    # List all the possible neighborhood sizes.
    n_nearest = [2, 3, 4, 5, 6, 7, 8, 9, 10]
    
    # Define the number of repetitions.
    N_REPEATS = 30
    
    # Maximum function evaluations.
    MAX_F_EVALS = 200_000

    # Set a fixed swarm size.
    SWARM_SIZE = 600
    
    # Stores the statistics.
    stats_per_run = defaultdict(list)
    
    # Set up the main loop.
    for n_neighbors in n_nearest:
    
        print(f"Running simulation with number of neighbors={n_neighbors} ... \n", flush=True)
    
        for k in range(N_REPEATS):
    
            print(f"Repetition {k} in process ... ", flush=True)
    
            # Get an initial (new) sample.
            x_sample = benchmark.sample_random_positions(n_pos=SWARM_SIZE)
    
            # Initial swarm population.
            swarm_t0 = Swarm([Particle(x) for x in x_sample])
        
            # Create the StandardPSO object that will carry on the optimization.
            test_PSO = StandardPSO(initial_swarm = swarm_t0, obj_func = func_benchmark,
                                   x_min = benchmark.x_min, x_max = benchmark.x_max)

            # Set the neighbors size.
            test_PSO.nn(n_neighbors)
            
            # Run the experiment.
            test_PSO.run(max_it = 500, f_max_eval = MAX_F_EVALS, verbose = False,
                         options = {"w0": 0.75, "c1": 1.50, "c2": 1.50, "mode": "multimodal"})
    
            # Obtain the number of the global optima that were found for epsilon = 1.0e-4.
            gopt_found, gopt_total = benchmark.search_for_optima(test_PSO.swarm.best_n(SWARM_SIZE))
    
            # Update the dictionary.
            stats_per_run[n_neighbors].append(gopt_found)
            
            # Leave a blank line between repetitions.
            print(" ")
        # _end_for_
    # _end_for_
    
    # Save the results.
    with open(f"E02_result_{benchmark.name}.json", 'w') as fp:
        json.dump(stats_per_run, fp)
    
    # Display final message.
    print(f"Experiment FINISHED.")

    # Get the data (counts).
    labels, data = stats_per_run.keys(), stats_per_run.values()

    # Prepare the PLOT.
    plt.boxplot(data)
    plt.xticks(range(1, len(labels) + 1), labels)
    
    plt.title(f"{benchmark.name}: No. Global optima = {gopt_total}")
    plt.xlabel("Number of nearest neighbors")
    plt.ylabel("Global optima found")
    plt.grid()
    
    # Save to file.
    plt.savefig(f"E02_{benchmark.name}.pdf", format="pdf", dpi=300)
# _end_if_

### End of file